Oskar Bujacz, obliczenia w Jupyterze w Julia 1.2.0, pracownia 2 z ANM, temat kwadratur

pracownia P2<br>
P2.14. 10 punktów Obliczyć przybliżoną wartość całki a f (x) dx przy założeniu, że znane są tylko wartości
f w zadanych z góry punktach a = t 0 < t 1 < · · · < t n = b. Wykonać obliczenia kontrolne, m. in., dla
następujących funkcji podcałkowych:

In [1]:
using Plotly

┌ Info: Recompiling stale cache file /home/oskar/.julia/compiled/v1.2/Plotly/M5iCk.ji for Plotly [58dd65bb-95f3-509e-9936-c39a10fdeae7]
└ @ Base loading.jl:1240
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-14781004051850080325\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-12753900043876738176\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-14781004051850080325\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [1]:
using LinearAlgebra
using QuadGK
import Distributions: Uniform 
using Statistics
using FFTW


In [442]:
#całkowanie przez próbkowanie prostokatami
#mozna zmodyfikowac do dzialania nie tylko na wezlach rownoodleglych
function int_rect(x,y)
    #sum = 0
    #for i =1:1000#int(length(y))
     #   sum += y[i]*len
    #end
    h = x[2]-x[1]
    return sum(y)*h #zakladamy wezly rownoodlegle
end

int_rect (generic function with 2 methods)

In [400]:
#metoda trapezów
function int_trap(x,y)
    len = size(x)[1]
    a = x[1]; b=x[len]#krance przedzialu
    h = x[2] - x[1]
    sum1 = 0.0
    sum1 = sum(y)
    sum1-= a/2 +b/2
    return sum1*h
end

int_trap (generic function with 1 method)

In [55]:
#metoda simpsona
function int_simpson(x,y)#dwie tablice, arg i wartosci
    len = size(x)[1]
    sum = 0.0
    for i = 2:trunc(Int,len/2)
       sum += 1/3*y[2*i-2] + 4/3*y[2*i-1] + 1/3*y[2*i] 
    end
    sum+=(y[1]+y[2])*(x[2]-x[1])/2#pierwszy przedzial
    return  sum*(x[2]-x[1])#h, zakladamy rownoodleglosc
end

int_simpson (generic function with 1 method)

In [83]:
#metoda 3/8
function int_newtonc3(x,y)
    len = size(x)[1]
    sum = 0.0
    for i = 1:trunc(Int,len/3-1)
        sum+= y[3*i-2] + 3*y[3*i-1] + 3*y[3*i] + y[3*i+1]
    end
    sum += y[1]+3*y[2]+3*y[3]+y[4]
    return sum*3/8*(x[2]-x[1])
end

int_newtonc3 (generic function with 1 method)

In [84]:
#simpson dla dowolnych wezlow
function int_simpson2(x,y)
    len = size(x)[1]
    h = x[2] - x[1]
    sum1, sum2 = 0.0, 0.0
    for i = 1:trunc(Int, len/2)
        sum1 += y[2*i-1]
        sum2 += y[2*i]
    end
    sum1 -= y[1]/2
    sum1 -= y[trunc(Int, len/2)]
    return h/3* (2*sum1 + 4*sum2)
end

int_simpson2 (generic function with 1 method)

In [404]:
function GaussLegendre(f,n)# w=1
    b = 1 ./ sqrt.(4.0.-(1:n).^(-2.0))
    #print(b)
    T = SymTridiagonal(zeros(n+1),b)
    x, V = eigvals(T), eigvecs(T)#wartosci wlasne, wektory wlasne
    w = 2.0*vec(V[1,:]).^2
    return dot(w, f.(x))
    #bedzie najlepszy, ale wolniejszy niz clenashw curtis
end

GaussLegendre (generic function with 1 method)

In [330]:
#testy wydajnosci
@time(GaussLegendre(x -> exp(-x^2), 10000))

 26.187097 seconds (510.49 k allocations: 791.719 MiB, 0.17% gc time)


1.493648265624855

In [53]:
@time(GaussLegendre(x -> exp(-x^2), 15000))

 71.769361 seconds (141.12 k allocations: 1.689 GiB, 0.30% gc time)


1.4936482656248435

In [657]:
X2 = collect(-1:0.01:1);
Y2 = exp.(-X2.^2);

In [48]:
# w = 1/sqrt(1-x^2) * pi/n
function GaussChebyshev(f,n)
    x = cos.((collect(1:2:(2*n-1))*pi)/(2n))
    return pi/(n)*sum(f.(x))
end
#w = 1/sqrt(1-x^2) * pi/n, wezly czebyshev ekstrema 
function Lobatto(f,n)
    x = cos.(collect(0:n)*pi/n)
    y = f.(x)
    y[1] *= 0.5; y[n+1]*=0.5
    return pi/n*sum(y)
end

GaussC (generic function with 1 method)

In [659]:
@show(GaussChebyshev(x -> exp(-x^2)*sqrt(1-x^2), 1000))
@show(Lobatto(x -> exp(-x^2)*sqrt(1-x^2), 1000))

GaussChebyshev((x->begin
            #= In[659]:1 =#
            exp(-(x ^ 2)) * sqrt(1 - x ^ 2)
        end), 1000) = 1.4936485681931313
Lobatto((x->begin
            #= In[659]:2 =#
            exp(-(x ^ 2)) * sqrt(1 - x ^ 2)
        end), 1000) = 1.4936476604879265


1.4936476604879265

In [25]:
integral, err = quadgk(x -> exp(-x^2), -1, 1, rtol=1e-8)

(1.493648265624854, 1.5774048733874224e-12)

In [140]:
#clenshaw curtis implemntacja 
function ClenshawC(f,n)
    x = cos.(collect(0:n)*pi/n)#pkt ekstremalne czebyszewa
    y = f.(x)
    #jawny wzor na Ak
    A = zeros(n+1)
    for k=1:n+1
        temp = 0.0
        for j=1:Int(n/2)-1#  bo sumbis 
            temp+=cos((k-1)*2*j*pi/n)/(1-4j*j)
        end
        temp+= (0.5*cos(0) + cos((k-1)*2*Int(n/2)*pi/n)*0.5/(1-4*Int(n/2)*Int(n/2)))
        A[k] = temp*4/n
    end
    s = sum(dot(A[2:n], y[2:n]))
    s += A[1]*y[1]*0.5 + A[n+1]*y[n+1]*0.5#pierwszy i ostatni
    return s

end

ClenshawC (generic function with 1 method)

In [141]:
@time(@show(ClenshawC(x -> exp(-x^2), 5000)))

ClenshawC((x->begin
            #= In[141]:1 =#
            exp(-(x ^ 2))
        end), 5000) = 1.493648265624854
  0.421989 seconds (463.41 k allocations: 24.280 MiB, 2.30% gc time)


1.493648265624854

In [155]:
#clenshaw curtis FFT 
function ClenshawCFast(f,n)
    x = cos.(collect(0:n)*pi/n)#pkt ekstremalne czebyszewa tu oki
    y = f.(x) ./ (2*n)
    vx = vcat(y,  y[2:n])
    A = zeros(n+1)
    g = real(myFFT(vx))
    A[1], A[n+1] = g[1], g[n+1]
    A[2:n] = g[2:n] + g[2*n:-1:n+2]
    w = 0*A';
    for i =1:2:n
         w[i] = 2/(1-(i-1)^2)
    end
    I = w*A; 
    return I

end

ClenshawCFast (generic function with 1 method)

In [156]:
@time(aaa = ClenshawCFast(x -> exp(-x^2), 32768))
bbb = quadgk(x -> exp(-x^2), BigFloat(-1), 1-eps(Float64), rtol=1e-16)[1]
print((bbb-aaa)/bbb)

  0.241824 seconds (921.40 k allocations: 133.925 MiB, 15.87% gc time)
-5.195784839613482864411319170164523322062724150739997350695675775770069925031906e-18

In [150]:
@time(aaa = ClenshawC(x -> exp(-x^2), 32768))
bbb = quadgk(x -> exp(-x^2), BigFloat(-1), 1-eps(Float64), rtol=1e-16)[1]
print((bbb-aaa)/bbb)

  9.741661 seconds (141.06 k allocations: 9.282 MiB, 0.07% gc time)
-5.195784839613482864411319170164523322062724150739997350695675775770069925031906e-18

In [160]:
time = zeros(10); w = 128
for i=1:11
    @time(ClenshawC(x -> exp(-x^2), w))
    w*=2
end

  0.046311 seconds (141.04 k allocations: 7.287 MiB)
  0.000741 seconds (27 allocations: 17.781 KiB)
  0.002991 seconds (28 allocations: 33.797 KiB)
  0.010233 seconds (28 allocations: 65.797 KiB)
  0.037903 seconds (34 allocations: 129.516 KiB)
  0.162543 seconds (36 allocations: 257.422 KiB)
  0.569408 seconds (36 allocations: 513.422 KiB)
  2.457818 seconds (36 allocations: 1.001 MiB, 0.19% gc time)
 10.145807 seconds (36 allocations: 2.001 MiB)
 41.925297 seconds (36 allocations: 4.001 MiB)
326.514293 seconds (36 allocations: 8.001 MiB)


In [161]:
w=128
for i=1:11
    @time(ClenshawCFast(x -> exp(-x^2), w))
    w*=2
end

  0.152239 seconds (198.24 k allocations: 10.435 MiB, 5.22% gc time)
  0.000497 seconds (4.64 k allocations: 681.797 KiB)
  0.001016 seconds (9.25 k allocations: 1.400 MiB)
  0.001706 seconds (18.47 k allocations: 2.940 MiB)
  0.003316 seconds (36.91 k allocations: 6.159 MiB)
  0.013275 seconds (73.80 k allocations: 12.878 MiB, 46.35% gc time)
  0.022323 seconds (147.56 k allocations: 26.879 MiB, 24.97% gc time)
  0.047356 seconds (295.09 k allocations: 56.007 MiB, 33.13% gc time)
  0.162157 seconds (590.15 k allocations: 116.512 MiB, 18.15% gc time)
  0.261693 seconds (1.18 M allocations: 242.022 MiB, 22.39% gc time)
  0.566005 seconds (2.36 M allocations: 502.043 MiB, 36.55% gc time)


In [170]:
w = 128
for i=1:11
    @time(ClenshawC(x -> g3(x), w))
    w*=2
end

  0.163924 seconds (146.97 k allocations: 7.523 MiB)
  0.001298 seconds (27 allocations: 17.781 KiB)
  0.004998 seconds (28 allocations: 33.797 KiB)
  0.019509 seconds (28 allocations: 65.797 KiB)
  0.077873 seconds (34 allocations: 129.516 KiB)
  0.315701 seconds (36 allocations: 257.422 KiB)
  1.199181 seconds (36 allocations: 513.422 KiB)
  4.910388 seconds (36 allocations: 1.001 MiB)
 20.311407 seconds (36 allocations: 2.001 MiB)
 84.607258 seconds (36 allocations: 4.001 MiB)
345.382358 seconds (36 allocations: 8.001 MiB, 0.00% gc time)


In [165]:
w=128
for i=1:11
    @time(ClenshawCFast(x -> g3(x), w))
    w*=2
end

  0.212718 seconds (332.54 k allocations: 17.162 MiB, 4.62% gc time)
  0.000461 seconds (4.64 k allocations: 681.797 KiB)
  0.000841 seconds (9.25 k allocations: 1.400 MiB)
  0.003473 seconds (18.47 k allocations: 2.940 MiB)
  0.005200 seconds (36.91 k allocations: 6.159 MiB)
  0.017666 seconds (73.80 k allocations: 12.878 MiB, 52.11% gc time)
  0.020526 seconds (147.56 k allocations: 26.879 MiB, 29.74% gc time)
  0.043239 seconds (295.09 k allocations: 56.007 MiB, 31.31% gc time)
  0.167805 seconds (590.15 k allocations: 116.512 MiB, 17.72% gc time)
  0.305183 seconds (1.18 M allocations: 242.022 MiB, 25.18% gc time)
  0.671152 seconds (2.36 M allocations: 502.043 MiB, 40.94% gc time)


In [167]:
fc(x) = 2.3535*cos(-5.76523*pi^x*(x+2.5231)\(0.82371*x+-0.1311))

fc (generic function with 1 method)

In [168]:
w = 128
for i=1:11
    @time(ClenshawC(x -> fc(x), w))
    w*=2
end

  0.128043 seconds (165.37 k allocations: 8.410 MiB)
  0.001246 seconds (27 allocations: 17.781 KiB)
  0.004679 seconds (28 allocations: 33.797 KiB)
  0.018808 seconds (28 allocations: 65.797 KiB)
  0.075688 seconds (34 allocations: 129.516 KiB)
  0.288640 seconds (36 allocations: 257.422 KiB)
  1.180448 seconds (36 allocations: 513.422 KiB)
  4.896094 seconds (36 allocations: 1.001 MiB)
 19.951047 seconds (36 allocations: 2.001 MiB, 0.06% gc time)
 83.672176 seconds (36 allocations: 4.001 MiB)
344.547595 seconds (36 allocations: 8.001 MiB)


In [169]:
w=128
for i=1:11
    @time(ClenshawCFast(x -> fc(x), w))
    w*=2
end

  0.174646 seconds (218.81 k allocations: 11.229 MiB, 3.53% gc time)
  0.000608 seconds (4.64 k allocations: 681.797 KiB)
  0.000922 seconds (9.25 k allocations: 1.400 MiB)
  0.003148 seconds (18.47 k allocations: 2.940 MiB)
  0.005372 seconds (36.91 k allocations: 6.159 MiB)
  0.014195 seconds (73.80 k allocations: 12.878 MiB, 47.21% gc time)
  0.026525 seconds (147.56 k allocations: 26.879 MiB, 29.47% gc time)
  0.057261 seconds (295.09 k allocations: 56.007 MiB, 34.52% gc time)
  0.174046 seconds (590.15 k allocations: 116.512 MiB, 18.24% gc time)
  0.441867 seconds (1.18 M allocations: 242.022 MiB, 46.13% gc time)
  0.758619 seconds (2.36 M allocations: 502.043 MiB, 33.87% gc time)


In [131]:
#dyskretna transformacja cosinusowa do O(nlogn)
#daje wszystkie Beta
function myFFT(x)
    N = size(x)[1]
    if (N==1) return x end
    #y = 0.0
    M = Int(floor(N/2))
    xL = x[1:M]
    xR = x[M+1:N]
    ye = myFFT(xL+xR)
    yo = myFFT((xL-xR).*[exp(Complex(0,2*pi*j/N)) for j=0:M-1])
    y = Complex(0,0) * zeros(N)
    y[1:2:N-1] = ye
    y[2:2:N] = yo
    return y
end

myFFT (generic function with 1 method)

In [1]:
#metoda romberga
function romberg(f,a,b,n)
    h = (b - a) ./ (2 .^ (0:n-1));
    r = zeros(n,n)
    r[1,1] = (b - a) * (f(a) + f(b)) / 2;#pierwszy trapez
    for j = 2:n
        s = 0;
        for i = 1:2^(j-2)#wykladniczo zwiekszamy ilosc wezlow
            s += f(a + (2 * i - 1) * h[j]);
        end
        r[j,1] = r[j-1,1] / 2 + h[j] * s;
        for k = 2:j#wzor reku na pozostale w wierszu
            r[j,k] = (4^(k-1) * r[j,k-1] - r[j-1,k-1]) / (4^(k-1) - 1);
        end
    end
    return r
end

romberg (generic function with 1 method)

In [2]:
@time(romberg(x -> exp(-x^2), -1,1,6))

  0.238828 seconds (605.65 k allocations: 30.833 MiB, 5.10% gc time)


6×6 Array{Float64,2}:
 0.735759  0.0      0.0      0.0      0.0      0.0    
 1.36788   1.57859  0.0      0.0      0.0      0.0    
 1.46274   1.49436  1.48875  0.0      0.0      0.0    
 1.48597   1.49371  1.49367  1.49375  0.0      0.0    
 1.49173   1.49365  1.49365  1.49365  1.49365  0.0    
 1.49317   1.49365  1.49365  1.49365  1.49365  1.49365

In [661]:
@time(romberg(x -> exp(-x^2), -1,1,6))

  0.194360 seconds (288.30 k allocations: 14.368 MiB, 12.48% gc time)


6×6 Array{Float64,2}:
 0.735759  0.0      0.0      0.0      0.0      0.0    
 1.36788   1.57859  0.0      0.0      0.0      0.0    
 1.46274   1.49436  1.48875  0.0      0.0      0.0    
 1.48597   1.49371  1.49367  1.49375  0.0      0.0    
 1.49173   1.49365  1.49365  1.49365  1.49365  0.0    
 1.49317   1.49365  1.49365  1.49365  1.49365  1.49365

In [655]:
#zwykly simpsons, zwraca tez midpoint i wartosc
function simpsons_1(f, a, fa, b, fb)
    m = (a + b) / 2
    fm = f(m)
    return (m, fm, abs(b - a) / 6 * (fa + 4 * fm + fb))
end
function simpsons_reku(f, a, fa, b, fb, eps, whole, m, fm)
    lm, flm, left  = simpsons_1(f, a, fa, m, fm)#wartosci na podprzedzialach
    rm, frm, right = simpsons_1(f, m, fm, b, fb)
    delta = left + right - whole
    if abs(delta) <= 15 * eps#dopuszczalny blad simpsona
        return left + right + delta / 15
    end
    return simpsons_reku(f, a, fa, m, fm, eps/2, left , lm, flm) +
           simpsons_reku(f, m, fm, b, fb, eps/2, right, rm, frm)
end
function ad_simpsons(f, a, b, eps)
    fa, fb = f(a), f(b)
    m, fm, whole = simpsons_1(f, a, fa, b, fb)
    return simpsons_reku(f, a, fa, b, fb, eps, whole, m, fm)
end


ad_simpsons (generic function with 1 method)

In [660]:
@time(ad_simpsons(x -> exp(-x^2), -1, 1, 1e-09))

  0.107531 seconds (110.29 k allocations: 5.600 MiB)


1.4936482656249894

In [440]:

#funckje to testow
#wielomiany
#trygo
#wykladnicze
#log jakis przesuniety
#np sin(1/x) jako jakis oscylacyjny kolo zera?
# z e^x jakis z pi
#ile testow? setki najlepiej np wielomiany 10 stopnia z randomowymi 
#wspolczynnikami [-100,100]
#ile wezlow dawac np podwajamy ilosc wezlow
#


f1(x) = x^2
#@show(GaussChebyshev(x -> f1(x)*sqrt(1-x^2), 20))
x1 = collect(-1:0.0001:1)
y1 = f1.(x1);

In [444]:
@show(int_rect(x1,y1))
@show(int_trap(x1,y1))
@show(int_simpson(x1,y1))
@show(int_newtonc3(x1,y1))

int_rect(x1, y1) = 0.6667666699999265
int_trap(x1, y1) = 0.6667666699999265
int_simpson(x1, y1) = 0.666466686665924
int_newtonc3(x1, y1) = 0.6667666166729256


0.6667666166729256

In [663]:
@show(GaussChebyshev(x -> exp(-x^2)*sqrt(1-x^2), 20))
@show(Lobatto(x -> exp(-x^2)*sqrt(1-x^2), 1000))
@show(ClenshawC(x -> exp(-x^2), 1000))
@show(GaussLegendre(x -> exp(-x^2), 1000))
@show(romberg(x -> exp(-x^2), -1,1,6)[6,6])
@show(ad_simpsons(x -> exp(-x^2), -1, 1, 1e-09))
integral, err = quadgk(x -> exp(-x^2), -1, 1, rtol=1e-12)

GaussChebyshev((x->begin
            #= In[663]:1 =#
            exp(-(x ^ 2)) * sqrt(1 - x ^ 2)
        end), 20) = 1.494401966092653
Lobatto((x->begin
            #= In[663]:2 =#
            exp(-(x ^ 2)) * sqrt(1 - x ^ 2)
        end), 1000) = 1.4936476604879265
ClenshawC((x->begin
            #= In[663]:3 =#
            exp(-(x ^ 2))
        end), 1000) = 1.493648265624854
GaussLegendre((x->begin
            #= In[663]:4 =#
            exp(-(x ^ 2))
        end), 1000) = 1.4936482656248522
(romberg((x->begin
            #= In[663]:5 =#
            exp(-(x ^ 2))
        end), -1, 1, 6))[6, 6] = 1.493648266788029
ad_simpsons((x->begin
            #= In[663]:6 =#
            exp(-(x ^ 2))
        end), -1, 1, 1.0e-9) = 1.4936482656249894


(1.493648265624854, 7.887024366937112e-13)

In [873]:
#funkcja testujaca zwracajaca bledy wzgledne wszystkich zaimplementowanych metod + czas
function test(f, n)
    diff = 2/n
    x1 = collect(-1:diff:1)
    y1 = f.(x1)
    R = zeros(10)
    time = zeros(10)
    time[1] = @time(R[1] = int_rect(x1,y1))
    time[2] = @time(R[2] = int_trap(x1,y1))
    time[3] = @time(R[3] = int_simpson(x1,y1))
    time[4] = @time(R[4] = int_newtonc3(x1,y1))
    time[5] = @time(R[5] = GaussChebyshev(x -> f(x)*sqrt(1-x^2), n))
    time[6] = @time(R[6] = Lobatto(x -> f(x)*sqrt(1-x^2), n))
    time[7] = @time(R[7] = ClenshawC(x -> f(x), n))
    time[8] = @time(R[8] = GaussLegendre(x -> f(x), n))
    time[9] = @time(R[9] = romberg(x -> f(x), -1,1,15)[15,15])
    time[10] = @time(R[10] = ad_simpsons(x -> f(x), -1, 1-eps(Float64), 1e-10))
    res = quadgk(x -> f(x), BigFloat(-1), 1-eps(Float64), rtol=1e-16)[1]
    #return (R .- res) ./ res#, time
    print(time)
    return time
end

test (generic function with 1 method)

In [883]:
b = rand(Uniform(-100,100),10)
fff(x) = (b[1]+x*(b[2]+x*(b[3]+x*(b[4]+x*(b[5]+x*(b[6]+x*(b[7]+x*(b[8]+x*(b[9]+x*b[10])))))))))
ggg(x) = b[1]*sin(b[2]*sin(b[5]*cos((x+b[3])/b[4])))
t = test(fff, 10000)

  0.000012 seconds (1 allocation: 16 bytes)
  0.000007 seconds (1 allocation: 16 bytes)
  0.000013 seconds (1 allocation: 16 bytes)
  0.000009 seconds (1 allocation: 16 bytes)
  0.044145 seconds (437.66 k allocations: 8.310 MiB)
  0.043702 seconds (437.71 k allocations: 8.311 MiB)
  0.922103 seconds (417.67 k allocations: 8.232 MiB, 1.06% gc time)
 12.699390 seconds (417.71 k allocations: 774.073 MiB, 0.05% gc time)
  0.034785 seconds (622.69 k allocations: 9.504 MiB)
[-119.25830828475979, -119.25830828475979, -119.23334910445385, -119.13187924878372, -119.24999407762763, -119.24999305162166, -119.24999373562568, -119.24999373562578, -119.24999373562537, 0.0]

10-element Array{Float64,1}:
 -119.25830828475979
 -119.25830828475979
 -119.23334910445385
 -119.13187924878372
 -119.24999407762763
 -119.24999305162166
 -119.24999373562568
 -119.24999373562578
 -119.24999373562537
    0.0             

In [575]:
integral, err = quadgk(x -> exp(-x^2), BigFloat(-1), 1, rtol=1e-20)

(1.493648265624854050798934872263706010713, 4.154344010739461307558110386644449836898e-21)

In [671]:
#r1, t1 = test(x -> exp(-x^2),10)
r2  = test(x -> exp(cos(sin(4*x+30))),10000)

  0.000006 seconds
  0.000005 seconds
  0.000014 seconds
  0.000010 seconds
  0.030333 seconds (1.15 k allocations: 458.016 KiB)
  0.035412 seconds (1.16 k allocations: 458.422 KiB)
  1.799400 seconds (1.13 k allocations: 690.688 KiB)
 25.367854 seconds (1.16 k allocations: 766.515 MiB, 0.16% gc time)
  0.000014 seconds (2 allocations: 496 bytes)
  0.051035 seconds (1.25 k allocations: 41.984 KiB)


10-element Array{BigFloat,1}:
  0.0001045881872740564263588016587682147790037
  0.0001045881872740564263588016587682147790037
 -0.0002091276345019984428986835532018811806044
 -0.0001556658867807991874640127534394407791317
  4.300766229467011697111882867498080414117e-09
 -8.601532516638350533393829848223591107073e-09
 -1.589682425829328529649639252496390143914e-16
  2.60232158026695641816700420220470467456e-16 
 -1.2647214712888008076571524904057171923e-05  
 -1.179760208042165689375844298498962416937e-06

In [672]:
r2

10-element Array{BigFloat,1}:
  0.0001045881872740564263588016587682147790037
  0.0001045881872740564263588016587682147790037
 -0.0002091276345019984428986835532018811806044
 -0.0001556658867807991874640127534394407791317
  4.300766229467011697111882867498080414117e-09
 -8.601532516638350533393829848223591107073e-09
 -1.589682425829328529649639252496390143914e-16
  2.60232158026695641816700420220470467456e-16 
 -1.2647214712888008076571524904057171923e-05  
 -1.179760208042165689375844298498962416937e-06

In [673]:
function test_poly(n)
    b = rand(Uniform(-100,100),10)
    res = zeros(10)
    res_t = zeros(10)
    #for i=1:m
    b = rand(Uniform(-100,100),10)
    f(x) = (b[1]+x*(b[2]+x*(b[3]+x*(b[4]+x*(b[5]+x*(b[6]+x*(b[7]+x*(b[8]+x*(b[9]+x*b[10])))))))))
    res = test(f,n)
    #end
    return res
end

test_poly (generic function with 3 methods)

In [775]:
function test_poly_mean(n,m)#testuje wielomiany st n z losowami wspolczynnikami m razy i zwrac
    #srednia bledy wzgledngo
    R = zeros(m,10)
    diff = 2/n
    S = zeros(10)
    x1 = collect(-1:diff:1)
    for i=1:m
        b = rand(Uniform(-100,100),10)
        f(x) = (b[1]+x*(b[2]+x*(b[3]+x*(b[4]+x*(b[5]+x*(b[6]+x*(b[7]+x*(b[8]+x*(b[9]+x*b[10])))))))))
        y1 = f.(x1)
        R[i,1] = int_rect(x1,y1)
        R[i,2] = int_trap(x1,y1)
        R[i,3] = int_simpson(x1,y1)
        R[i,4] = int_newtonc3(x1,y1)
        R[i,5] = GaussChebyshev(x -> f(x)*sqrt(1-x^2), n)
        R[i,6] = Lobatto(x -> f(x)*sqrt(1-x^2), n)
        R[i,7] = ClenshawC(x -> f(x), n)
        R[i,8] = GaussLegendre(x -> f(x), n)
        R[i,9] = romberg(x -> f(x), -1,1,6)[6,6]
        R[i,10] = ad_simpsons(x -> f(x), -1, 1, 1e-10)
        res = quadgk(x -> f(x), BigFloat(-1), 1, rtol=1e-16)[1]
        R[i,:] = (R[i,:] .- res) ./ res
    end
    
    for i=1:10
         S[i] = abs(mean(R[:,i]))
    end
    return S#(R .- res) ./ res 
    
end

test_poly_mean (generic function with 1 method)

In [727]:
p1 = test_poly_mean(10,100)
p2 = test_poly_mean(100,100)
p3 = test_poly_mean(1000,100)
#p4 = test_poly_mean(10000,50)
#p5 = test_poly_mean(50000,10)
#p6 = test_poly_mean(1000,50)

10-element Array{Float64,1}:
 0.0002533202112023259 
 0.0002533202112023259 
 0.0004946671949417865 
 0.03330790205149437   
 1.0355996005928518e-7 
 2.0711852798292752e-7 
 6.453085391630486e-17 
 1.0291171573115754e-15
 4.1082644033040185e-17
 5.853889799723462e-17 

In [745]:
using Printf
for i=1:10
    @printf("%e \n",p4[i])
end

2.453698e-04 
2.453698e-04 
4.904801e-04 
3.791611e-04 
1.008909e-08 
2.017819e-08 
2.096803e-16 
7.994914e-16 
5.028346e-17 
7.163252e-17 


In [732]:

@time(p4 = test_poly_mean(10000,10))

271.160453 seconds (433.55 k allocations: 7.506 GiB, 0.21% gc time)


10-element Array{Float64,1}:
 0.00024536975245574035
 0.00024536975245574035
 0.0004904801160573938 
 0.000379161059405947  
 1.0089093406810636e-8 
 2.0178185974227104e-8 
 2.0968028817874794e-16
 7.994914088681734e-16 
 5.028346070465163e-17 
 7.16325246164961e-17  

In [674]:
a = test_poly(1000)


  0.000017 seconds (1 allocation: 16 bytes)
  0.000003 seconds (1 allocation: 16 bytes)
  0.000004 seconds (1 allocation: 16 bytes)
  0.000003 seconds (1 allocation: 16 bytes)
  0.065133 seconds (82.92 k allocations: 2.768 MiB)
  0.063755 seconds (82.97 k allocations: 2.769 MiB)
  0.077511 seconds (81.08 k allocations: 2.765 MiB)
  0.207429 seconds (81.11 k allocations: 10.687 MiB)
  0.000063 seconds (1.27 k allocations: 20.234 KiB)
  0.073240 seconds (74.16 k allocations: 3.880 MiB, 16.08% gc time)


10-element Array{BigFloat,1}:
  0.001742139646461468905414585812050132460351 
  0.001742139646461468905414585812050132460351 
 -0.003432442331147911783144133396721496380326 
  0.1382936934867039656206824418254511437303   
  7.137564083999523018085226874984035256421e-07
 -1.427506695373842398029507771899121051206e-06
  4.488808491246726910367059247588494055559e-17
 -1.043802760714562169657472468378258442866e-14
  6.543599274739840694338098527234703689994e-16
  2.502189863777973456110017553341207739804e-09

In [162]:
Pi = 3.1415926535979323
g1(x) = 1/(x^4+x^2+0.9)
g2(x) = 4/(Pi*(1+x^2))
g3(x) = (2*pi^2*(1+x))/((1-eps(Float64)-x)*(3+x))*sin(pi*(1+x))

g3 (generic function with 1 method)

In [625]:
xg3 = collect(-1:0.01:1)
yg3 = g3.(xg3)
plot(xg3,yg3)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with field margin"

In [715]:
ad_simpsons(x -> g3(x), -1, 1-eps(Float64), 1e-10)

-14.727040999163346

In [767]:
g11 = test(g1,100)

  0.000001 seconds
  0.000001 seconds
  0.000001 seconds
  0.000001 seconds
  0.030665 seconds (1.15 k allocations: 71.359 KiB)
  0.030071 seconds (1.16 k allocations: 71.422 KiB)
  0.025804 seconds (1.13 k allocations: 71.656 KiB)
  0.030202 seconds (1.15 k allocations: 181.781 KiB)
  0.000009 seconds (2 allocations: 496 bytes)


10-element Array{BigFloat,1}:
  0.004328685903720686567474672115075454239082 
  0.004328685903720686567474672115075454239082 
 -0.008900028114476693322954023811440920796533 
 -0.005030357940425688023918168308307171498372 
  1.792193844997724614108768249317814092481e-05
 -3.584617942817942283143614581690313262162e-05
  8.464236618115567751249220944732439222416e-17
 -1.45905613471333581408660630540243595007e-15 
 -2.675358224007269002011601234355202063484e-08
 -1.0                                          

In [752]:
g12 = test(g1,10000)

  0.000006 seconds
  0.000006 seconds
  0.000019 seconds
  0.000017 seconds
  0.002063 seconds (15 allocations: 391.094 KiB)
  0.001762 seconds (21 allocations: 391.500 KiB)
  1.806810 seconds (32 allocations: 626.266 KiB)
 25.522280 seconds (66 allocations: 766.452 MiB, 0.20% gc time)
  0.000022 seconds (2 allocations: 496 bytes)
  0.000735 seconds (6.87 k allocations: 115.156 KiB)


10-element Array{BigFloat,1}:
  4.358445463161530400214526695523555087878e-05
  4.358445463161530400214526695523555087878e-05
 -8.719296001868023663530686796981589425125e-05
 -4.365062769788931073756209277819405431443e-05
  1.792462651895230541687668609276480319241e-09
 -3.584924909527135185863623626612830946528e-09
 -1.960300885269336845964347932526320336479e-16
  6.45987275597334401730346214847237243967e-16 
 -2.675358224007269002011601234355202063484e-08
  2.751030685908004617547298735096910438009e-15

In [759]:
using Printf
for i=1:10
    @printf("%e \n",g22[i])
end

6.365986e-05 
6.365986e-05 
-1.273367e-04 
-6.370656e-05 
2.617994e-09 
-5.235988e-09 
-4.626746e-17 
8.419110e-16 
6.938956e-09 
5.726892e-15 


In [757]:
g21 = test(g2,100)
g22 = test(g2,10000)

  0.000004 seconds (1 allocation: 16 bytes)
  0.000004 seconds (1 allocation: 16 bytes)
  0.000003 seconds (1 allocation: 16 bytes)
  0.000003 seconds (1 allocation: 16 bytes)
  0.066707 seconds (43.78 k allocations: 2.145 MiB)
  0.066216 seconds (43.79 k allocations: 2.133 MiB)
  0.075540 seconds (43.55 k allocations: 2.130 MiB, 25.37% gc time)
  0.058450 seconds (43.57 k allocations: 2.240 MiB)
  0.000024 seconds (159 allocations: 2.938 KiB)
  0.061006 seconds (79.76 k allocations: 3.964 MiB)
  0.000007 seconds (1 allocation: 16 bytes)
  0.000004 seconds (1 allocation: 16 bytes)
  0.000016 seconds (1 allocation: 16 bytes)
  0.000012 seconds (1 allocation: 16 bytes)
  0.001752 seconds (59.51 k allocations: 1.290 MiB)
  0.001754 seconds (59.52 k allocations: 1.290 MiB)
  1.778013 seconds (39.53 k allocations: 1.214 MiB)
 25.645901 seconds (39.56 k allocations: 767.055 MiB, 0.18% gc time)
  0.000027 seconds (159 allocations: 2.938 KiB)
  0.001060 seconds (12.41 k allocations: 201.766 Ki

10-element Array{BigFloat,1}:
  6.365985506063474015252844137480563489637e-05
  6.365985506063474015252844137480563489637e-05
 -0.0001273366878246515972050050801563302236792
 -6.370655643593885141379661209173723701984e-05
  2.617993775468989011600274420157567659629e-09
 -5.235987689740367153546846361654799257883e-09
 -4.626746304344876584044655464620375534662e-17
  8.419109566589775326652322234386450676577e-16
  6.938956474236331434680894263695181765351e-09
  5.726892265022322174446465502905313594168e-15

In [874]:
g31 = test(g3,100)
res = quadgk(x -> g3(x), BigFloat(-1), 0.999999999999999, rtol=1e-20)[1]
g31[10] = ad_simpsons(x -> g3(x), -1, 0.999999999999999, 1e-10)

  0.000005 seconds (1 allocation: 16 bytes)
  0.000001 seconds (1 allocation: 16 bytes)
  0.000001 seconds (1 allocation: 16 bytes)
  0.000001 seconds (1 allocation: 16 bytes)
  0.022112 seconds (1.19 k allocations: 73.953 KiB)
  0.024036 seconds (1.20 k allocations: 74.016 KiB)
  0.015787 seconds (1.17 k allocations: 74.422 KiB)
  0.019074 seconds (1.19 k allocations: 184.547 KiB)
  0.000209 seconds (2 allocations: 2.188 KiB)
[-14.199500572851898, -14.199500572851898, -14.10864768845002, -12.279122276393865, -14.728316145358225, -14.724490761774184, -14.722851272215095, -14.727040999163396, -14.725483433400408, 0.0]

-14.72704099916356

In [521]:
b = rand(Uniform(-100,100),10)
f(x) = (b[1]+x*(b[2]+x*(b[3]+x*(b[4]+x*(b[5]+x*(b[6]+x*(b[7]+x*(b[8]+x*(b[9]+x*b[10])))))))))
@time(GaussLegendre(x -> f(x), 10))

  0.121456 seconds (119.50 k allocations: 6.246 MiB, 11.76% gc time)


-0.31861487919167253

In [875]:
g32 = test(g3, 10000)
g32[10]=g31[10]


  0.000014 seconds (1 allocation: 16 bytes)
  0.000014 seconds (1 allocation: 16 bytes)
  0.000055 seconds (1 allocation: 16 bytes)
  0.000044 seconds (1 allocation: 16 bytes)
  0.001359 seconds (15 allocations: 391.094 KiB)
  0.001099 seconds (21 allocations: 391.500 KiB)
  0.884721 seconds (32 allocations: 626.266 KiB)
 13.353339 seconds (66 allocations: 766.452 MiB, 0.34% gc time)
  0.000559 seconds (2 allocations: 2.188 KiB)
[-14.721763036717228, -14.721763036717228, -14.720839899039545, -14.702238466237409, -14.727041126671711, -14.727040744147034, -14.72704058023264, -14.727040999163483, -14.725483433400408, 0.0]

-14.72704099916356

In [801]:
for i=1:10
    @printf("%e \n",(res - g31[i])/res)
end

3.582121e-02 
3.582121e-02 
4.199033e-02 
1.662193e-01 
-8.658536e-05 
1.731670e-04 
2.844921e-04 
5.691906e-15 
1.057623e-04 
-5.405016e-15 


In [827]:
function test_try1_mean(n,m)#testuje wielomiany st n z losowami wspolczynnikami m razy i zwrac
    #srednia bledy wzgledngo
    R = zeros(m,10)
    diff = 2/n
    S = zeros(10)
    x1 = collect(-1:diff:1)
    for i=1:m
        b = rand(Uniform(-100,100),5)
        #f(x) = b[1]*sin(b[2]*sin(b[5]*cos((x+b[3])/b[4])))
        e = 2.718281828459045
        f(x) = b[1]*cos(b[2]*e^x*(x+b[3])\(b[4]*x+b[5]))
        y1 = f.(x1)
        R[i,1] = int_rect(x1,y1)
        R[i,2] = int_trap(x1,y1)
        R[i,3] = int_simpson(x1,y1)
        R[i,4] = int_newtonc3(x1,y1)
        R[i,5] = GaussChebyshev(x -> f(x)*sqrt(1-x^2), n)
        R[i,6] = Lobatto(x -> f(x)*sqrt(1-x^2), n)
        R[i,7] = ClenshawC(x -> f(x), n)
        R[i,8] = GaussLegendre(x -> f(x), n)
        R[i,9] = romberg(x -> f(x), -1,1,20)[20,20]
        R[i,10] = ad_simpsons(x -> f(x), -1, 1, 1e-10)
        res = quadgk(x -> f(x), BigFloat(-1), 1, rtol=1e-16)[1]
        R[i,:] = (R[i,:] .- res) ./ res
        print(i)
    end
    
    for i=1:10
         S[i] = abs(mean(R[:,i]))
    end
    return S#(R .- res) ./ res 
    
end

test_try1_mean (generic function with 1 method)

In [814]:
h1 = test_try1_mean(100,100)

10-element Array{Float64,1}:
 0.7117418713757426    
 0.7117418713757426    
 0.5740518966863803    
 0.7978337944091685    
 0.25889794037760283   
 0.3204052152070518    
 0.3200403668661106    
 0.3257299429540471    
 2.2694440577371494e-15
 2.0331796118154525e-12

In [815]:
@time(h2 = test_try1_mean(1000,100))

 20.387026 seconds (101.04 M allocations: 2.729 GiB, 4.81% gc time)


10-element Array{Float64,1}:
 0.037332808191895876  
 0.037332808191895876  
 0.02614216350494503   
 0.17314697198872167   
 0.04810829072263953   
 0.002843296407208815  
 0.0028263410924828924 
 0.013890147429456106  
 1.7247229631378204e-14
 5.1780234294428684e-15

In [816]:
@time(h3 = test_try1_mean(10000,5))

 74.120160 seconds (13.85 M allocations: 4.023 GiB, 1.10% gc time)


10-element Array{Float64,1}:
 0.004240346491455621  
 0.004240346491455621  
 0.008050356726221602  
 0.0019594440935443115 
 1.7295849944235016e-7 
 3.459099527648893e-7  
 1.1393507841154674e-12
 4.091468769673159e-13 
 9.182785004452063e-14 
 1.3431875117299266e-13

In [821]:
for i=1:10
    @printf("& %e  & %e \\\\ \n",h1[i], h3[i])
end

& 7.117419e-01  & 4.240346e-03 \\ 
& 7.117419e-01  & 4.240346e-03 \\ 
& 5.740519e-01  & 8.050357e-03 \\ 
& 7.978338e-01  & 1.959444e-03 \\ 
& 2.588979e-01  & 1.729585e-07 \\ 
& 3.204052e-01  & 3.459100e-07 \\ 
& 3.200404e-01  & 1.139351e-12 \\ 
& 3.257299e-01  & 4.091469e-13 \\ 
& 2.269444e-15  & 9.182785e-14 \\ 
& 2.033180e-12  & 1.343188e-13 \\ 


In [828]:
@time(hh1 = test_try1_mean(100,100))
#@time(hh2 = test_try1_mean(10000,7))

123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100  5.316740 seconds (3.88 M allocations: 174.746 MiB, 2.53% gc time)


10-element Array{Float64,1}:
 0.00949910104759187   
 0.00949910104759187   
 0.019097540456803716  
 0.026979724355124225  
 3.9107283458128894e-5 
 7.821457413117699e-5  
 1.3596552287094453e-17
 2.4492422287815494e-16
 7.099395647630109e-16 
 1.0724693309251078e-12

In [846]:
@time(hh2 = test_try1_mean(10000,5))

12345 76.107663 seconds (43.23 k allocations: 3.751 GiB, 0.76% gc time)


10-element Array{Float64,1}:
 9.915220384798565e-5  
 9.915220384798565e-5  
 0.00019830542456892816
 0.00010757155349531893
 4.077476327843321e-9  
 8.154952334258124e-9  
 1.51133989450985e-16  
 8.667171666216006e-16 
 1.4952426108436944e-15
 2.7677134611332656e-16

In [823]:
for i=1:10
    @printf("& %e  & %e \\\\ \n",hh1[i], hh2[i])
end

ℯ = 2.7182818284590...

In [858]:
function test_tryg2_mean(n,m)#testuje wielomiany st n z losowami wspolczynnikami m razy i zwrac
    #srednia bledy wzgledngo
    R = zeros(m,10)
    diff = 2/n
    S = zeros(10)
    x1 = collect(-1:diff:1)
    for i=1:m
        b = rand(Uniform(-100,100),5)
        #f(x) = b[1]*sin(b[2]*sin(b[5]*cos((x+b[3])/b[4])))
        e = 2.718281828459045
        f(x) = b[1]*(cos(b[2]))^2+sin(b[5]*tan(x))
        y1 = f.(x1)
        R[i,1] = int_rect(x1,y1)
        R[i,2] = int_trap(x1,y1)
        R[i,3] = int_simpson(x1,y1)
        R[i,4] = int_newtonc3(x1,y1)
        R[i,5] = GaussChebyshev(x -> f(x)*sqrt(1-x^2), n)
        R[i,6] = Lobatto(x -> f(x)*sqrt(1-x^2), n)
        R[i,7] = ClenshawC(x -> f(x), n)
        R[i,8] = GaussLegendre(x -> f(x), n)
        R[i,9] = romberg(x -> f(x), -1,1,20)[20,20]
        R[i,10] = ad_simpsons(x -> f(x), -1, 1, 1e-10)
        res = quadgk(x -> f(x), BigFloat(-1), 1, rtol=1e-16)[1]
        R[i,:] = (R[i,:] .- res) ./ res
        print(i)
    end
    
    for i=1:10
         S[i] = abs(mean(R[:,i]))
    end
    return S#(R .- res) ./ res 
    
end

test_tryg2_mean (generic function with 1 method)

In [860]:
@time(kk1 = test_tryg2_mean(100,100))


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100  2.656904 seconds (79.95 k allocations: 15.816 MiB, 0.76% gc time)


10-element Array{Float64,1}:
 0.010000000000000864  
 0.010000000000000864  
 0.019999999999999237  
 0.005076973875475204  
 4.112453549261992e-5  
 8.224805627902643e-5  
 8.812342286097328e-16 
 1.0492501931778205e-15
 4.292082627747932e-16 
 9.71175188754775e-17  

In [861]:
@time(kk2 = test_tryg2_mean(10000,5))


12345 70.164759 seconds (4.41 k allocations: 3.750 GiB, 0.83% gc time)


10-element Array{Float64,1}:
 9.999999988977612e-5  
 9.999999988977612e-5  
 0.00020000000011016224
 0.00038077616308129634
 4.112335274614987e-9  
 8.224670376838728e-9  
 1.824612653499666e-16 
 5.776259981153287e-16 
 2.2203484367214217e-15
 3.0099429194854814e-17

In [863]:
for i=1:10
    @printf("%e  & %e \\\\ \n",kk1[i], kk2[i])
end

1.000000e-02  & 1.000000e-04 \\ 
1.000000e-02  & 1.000000e-04 \\ 
2.000000e-02  & 2.000000e-04 \\ 
5.076974e-03  & 3.807762e-04 \\ 
4.112454e-05  & 4.112335e-09 \\ 
8.224806e-05  & 8.224670e-09 \\ 
8.812342e-16  & 1.824613e-16 \\ 
1.049250e-15  & 5.776260e-16 \\ 
4.292083e-16  & 2.220348e-15 \\ 
9.711752e-17  & 3.009943e-17 \\ 


In [869]:
function test_exp2_mean(n,m)#testuje wielomiany st n z losowami wspolczynnikami m razy i zwrac
    #srednia bledy wzgledngo
    R = zeros(m,10)
    diff = 2/n
    S = zeros(10)
    x1 = collect(-1:diff:1)
    for i=1:m
        b = rand(Uniform(-100,100),5)
        #f(x) = b[1]*sin(b[2]*sin(b[5]*cos((x+b[3])/b[4])))
        e = 2.718281828459045
        #f(x) = b[1]*e^x+b[2]* 2 .^ x + abs(b[3]/10.)^x
        f(x) = Pi^x*b[1]+abs(b[2])*x^5+abs(b[3]/10)^x
        y1 = f.(x1)
        R[i,1] = int_rect(x1,y1)
        R[i,2] = int_trap(x1,y1)
        R[i,3] = int_simpson(x1,y1)
        R[i,4] = int_newtonc3(x1,y1)
        R[i,5] = GaussChebyshev(x -> f(x)*sqrt(1-x^2), n)
        R[i,6] = Lobatto(x -> f(x)*sqrt(1-x^2), n)
        R[i,7] = ClenshawC(x -> f(x), n)
        R[i,8] = GaussLegendre(x -> f(x), n)
        R[i,9] = romberg(x -> f(x), -1,1,20)[20,20]
        R[i,10] = ad_simpsons(x -> f(x), -1, 1, 1e-10)
        res = quadgk(x -> f(x), BigFloat(-1), 1, rtol=1e-16)[1]
        R[i,:] = (R[i,:] .- res) ./ res
        print(i)
    end
    
    for i=1:10
         S[i] = abs(mean(R[:,i]))
    end
    return S#(R .- res) ./ res 
    
end

test_exp2_mean (generic function with 1 method)

In [843]:
@time(ee1 = test_exp_mean(100,100))


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100  3.388786 seconds (1.16 M allocations: 35.075 MiB, 0.23% gc time)


10-element Array{Float64,1}:
 0.013163685966107414  
 0.013163685966107414  
 0.026062448926277838  
 0.07923878950050228   
 5.400151864420229e-5  
 0.00010799865987255802
 3.0324308482567623e-17
 6.473173323248282e-19 
 1.2278419732634293e-15
 3.50532885563669e-17  

In [844]:
@time(ee2 = test_exp_mean(10000,5))


12345 75.949188 seconds (65.38 k allocations: 3.751 GiB, 0.78% gc time)


10-element Array{Float64,1}:
 0.00013130686177308855
 0.00013130686177308855
 0.0002625870589598054 
 0.0008309053628723698 
 5.399641342317235e-9  
 1.0799282440033345e-8 
 2.3200202422744796e-16
 9.60786433996643e-16  
 2.300136463820056e-15 
 6.987940773769261e-18 

In [848]:
for i=1:10
    @printf("%e  & %e \\\\ \n",ee1[i], ee2[i])
end

1.316369e-02  & 1.313069e-04 \\ 
1.316369e-02  & 1.313069e-04 \\ 
2.606245e-02  & 2.625871e-04 \\ 
7.923879e-02  & 8.309054e-04 \\ 
5.400152e-05  & 5.399641e-09 \\ 
1.079987e-04  & 1.079928e-08 \\ 
3.032431e-17  & 2.320020e-16 \\ 
6.473173e-19  & 9.607864e-16 \\ 
1.227842e-15  & 2.300136e-15 \\ 
3.505329e-17  & 6.987941e-18 \\ 


In [870]:
@time(ff1 = test_exp2_mean(100,100))


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100 21.303816 seconds (480.33 M allocations: 7.257 GiB, 10.08% gc time)


10-element Array{Float64,1}:
 0.014573500045085225  
 0.014573500045085225  
 0.02874861130289836   
 0.3648771536047502    
 5.973183812604862e-5  
 0.00011945761857020355
 1.9144952715820548e-17
 1.0117588807774198e-15
 2.618418285546018e-15 
 1.0722569956022802e-16

In [871]:
@time(ff2 = test_exp2_mean(10000,5))


12345 71.427534 seconds (26.30 M allocations: 4.142 GiB, 1.75% gc time)


10-element Array{Float64,1}:
 0.00014719358440241472
 0.00014719358440241472
 0.0002943446671903846 
 0.0024909423936833363 
 6.052875343725645e-9  
 1.2105750270198432e-8 
 1.4815926327643204e-16
 7.904739190139054e-16 
 2.1317179563523437e-15
 1.3838222266987357e-16

In [872]:
for i=2:10
    @printf("%e  & %e \\\\ \n",ff1[i], ff2[i])
end

1.457350e-02  & 1.471936e-04 \\ 
2.874861e-02  & 2.943447e-04 \\ 
3.648772e-01  & 2.490942e-03 \\ 
5.973184e-05  & 6.052875e-09 \\ 
1.194576e-04  & 1.210575e-08 \\ 
1.914495e-17  & 1.481593e-16 \\ 
1.011759e-15  & 7.904739e-16 \\ 
2.618418e-15  & 2.131718e-15 \\ 
1.072257e-16  & 1.383822e-16 \\ 
